# CNN MNIST 분류

## 모델 연습

In [1]:
import torch
import torch.nn as nn

inputs = torch.Tensor(1, 1, 28, 28) # 배치, 채널, 높이, 너비
inputs.shape

torch.Size([1, 1, 28, 28])

In [2]:
conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1)
conv1

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

In [3]:
conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
conv2

Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

In [4]:
pool = nn.MaxPool2d(2)
pool

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)

In [10]:
out = conv1(inputs)
out.shape

torch.Size([1, 32, 28, 28])

In [11]:
out = pool(out)
out.shape

torch.Size([1, 32, 14, 14])

In [12]:
out = conv2(out)
out.shape

torch.Size([1, 64, 14, 14])

In [13]:
out = pool(out)
out.shape

torch.Size([1, 64, 7, 7])

In [14]:
out.size(0), out.size(1), out.size(2), out.size(3) # n번째 차원 접근

(1, 64, 7, 7)

In [17]:
out = out.view(out.size(0), -1) # 텐서를 펼침
out.shape

torch.Size([1, 3136])

In [18]:
fc = nn.Linear(3136, 10)
out = fc(out)
out.shape

torch.Size([1, 10])

# CNN으로 MNIST 분류

In [19]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [26]:
torch.manual_seed(0)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device == 'cuda':
    torch.cuda.manual_seed_all(0)

device

'cpu'

In [27]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [28]:
mnist_train = dsets.MNIST(
    root='MNIST_data/',
    train=True,
    transform=transforms.ToTensor(),
    download=True
)
mnist_test = dsets.MNIST(
    root='MNIST_data/',
    train=False,
    transform=transforms.ToTensor(),
    download=True
)

In [29]:
data_loader = torch.utils.data.DataLoader(
    dataset=mnist_train,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True
)

In [30]:
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)
        
        torch.nn.init.xavier_uniform(self.fc.weight)
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [32]:
model = CNN().to(device)
model

C:\Users\user\AppData\Local\Temp\ipykernel_8952\2195289085.py:19: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(self.fc.weight)


CNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=3136, out_features=10, bias=True)
)

In [34]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
total_batch = len(data_loader)
total_batch

600

In [36]:
for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, y in data_loader:
        X = X.to(device)
        y = y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
    
    print(f'[epoch: {epoch:>4}] cost = {avg_cost:>.9}')

[epoch:    0] cost = 0.219247922
[epoch:    1] cost = 0.0596846044
[epoch:    2] cost = 0.0436499901
[epoch:    3] cost = 0.0345359705
[epoch:    4] cost = 0.0296303984
[epoch:    5] cost = 0.0231995061
[epoch:    6] cost = 0.0196018554
[epoch:    7] cost = 0.0169158522
[epoch:    8] cost = 0.0135247149
[epoch:    9] cost = 0.0119732441
[epoch:   10] cost = 0.00938998442
[epoch:   11] cost = 0.00844649225
[epoch:   12] cost = 0.00721900398
[epoch:   13] cost = 0.0063084336
[epoch:   14] cost = 0.00618505292
